# Fluxonium with multi-path coupling(Simultaneous CNOT gate)

This notebook is similar but not as detail as `examples/fluxonium_multipath_coupling/fluxonium_mpcoupling_6q_x.ipynb`. Please check that notebook for more info.

This example is based on Nguyen, L. B. et al. Blueprint for a High-Performance Fluxonium Quantum Processor. PRX Quantum 3, 037001 (2022).
We simulate a 6 fluxonium qubit system from an underlying periodic lattice.
Idling hamiltonian of the system is
$$ H(0) = \sum_i H_{\mathrm{f},i}  + \sum_{\langle i,j \rangle } H_{ij}. $$
Hamiltonian of single fluxonium is
$$ H_{\mathrm{f},i} = 4E_{\mathrm{C},i} n_i^2 + \frac{1}{2} E_{\mathrm{L},i} (\varphi_i +\varphi_{\mathrm{ext},i})^2
    -E_{\mathrm{J},i}\cos \left( \varphi_i \right).$$
The coupling terms have the form
$$H_{ij} = J_{\mathrm{C}} n_i n_j - J_{\mathrm{L}} \varphi_i \varphi_j.$$
The couplings are chosen in a way such that the idling $ZZ$-crosstalk is almost zero.

In [1]:
import jax
import jax.numpy as jnp
import haiku as hk

from supergrad.helper import Evolve
from supergrad.utils import tensor, permute, compute_fidelity_with_1q_rotation_axis
from supergrad.utils.optimize import scipy_minimize, adam_opt
from supergrad.utils.gates import cnot
from supergrad.utils.utility import tree_print

from supergrad.scgraph.graph_mpc_fluxonium_5x5_periodic import CNOTGatePeriodicGraphOpt

truncated_dim = 3  # how many levels we keep after diagonalization
add_random = True  # whether to add random deviations to fluxonium parameters ec,ej,el
share_params = True  # whether we treat marked parameters as shared when computing gradients
unify_coupling = True  # set all couplings to be equal

compensation_option = 'arbit_single'  # allow arbitary single-qubit gate compensation
# This is useful for checking how many errors are fundamentally multi-qubit

# This specify one order of the qubits such that the simultaneous gates are [cnot()] * 3
unitary_order = ['q02', 'q03', 'q12', 'q13', 'q22', 'q23']

# instance the quantum processor graph, and choose a subset for time evolution
gate_graph = CNOTGatePeriodicGraphOpt(seed=1)
qubit_subset = gate_graph.subgraph(unitary_order)
opt = 'adam'

evo = Evolve(qubit_subset, truncated_dim, add_random, share_params, unify_coupling, compensation_option)

# Compute the order of qubits in qubit_subset related to unitary_order
qubit_order = [unitary_order.index(key) for key in qubit_subset.sorted_nodes]
# Compute the corresponding target unitary based on the above order
target_unitary = permute(tensor(*([cnot()] * 3)), [2] * len(qubit_subset.nodes), qubit_order)


Let us print the parameters stored in the evo instance. These parameters are parsed from gate_graph. For how we construct this graph, please check `examples/fluxonium_multipath_coupling/graph_5x5_periodic.py` and the instructions inside.

In [2]:
params = evo.all_params
tree_print(params)


{ 'capacitive_coupling_all_unify': {'strength': 0.07225663},
  'inductive_coupling_all_unify': {'strength': -0.01256637},
  'q02': {'ec': 6.3532393823, 'ej': 25.1103087675, 'el': 6.344787656},
  'q02_pulse_rampcos': { 'amp': 0.18128846,
                         'omega_d': 2.58934559,
                         'phase': -0.24290228,
                         't_plateau': 69.93608145,
                         't_ramp': 29.92806488},
  'q03': {'ec': 6.2550610547000005, 'ej': 25.1411401038, 'el': 5.000964059},
  'q12': {'ec': 6.2233456874, 'ej': 25.1512697752, 'el': 5.6012793457},
  'q12_pulse_rampcos': { 'amp': 0.17872194,
                         'omega_d': 4.19989714,
                         'phase': -0.01543561,
                         't_plateau': 69.95327862,
                         't_ramp': 29.94562879},
  'q13': {'ec': 6.2867872105, 'ej': 25.1316550509, 'el': 6.9144529962},
  'q22': { 'ec': 6.282846661600001,
           'ej': 25.1329965509,
           'el': 7.539286440900001},
  '

For optimization, let us use an object function based on the average gate fidelity with leakage. The formula is from Physical Review A 87, 022309 (2013).
$$
	C = 1 - \frac{1}{D(D + 1)}[Tr(PU^{\dagger}_{\text{sim}}P U_{\text{sim}}P)+|Tr(PU^{\dagger}_{\text{sim}} P U_{\text{target}})|^2],   
$$
where $P$ is the projector into the computational subspace $S$, and $D=2^n$ is the dimension of $S$.

In [3]:
def infidelity(params, target_unitary):
    params = hk.data_structures.merge(evo.all_params, params)
    # Compute the time evolution unitary in the eigenbasis.
    sim_u = evo.eigen_basis(params)
    # calculate fidelity
    fidelity_vz, _ = compute_fidelity_with_1q_rotation_axis(target_unitary,
                                                            sim_u,
                                                            opt_method=None)

    return jnp.log10(1 - fidelity_vz)


In [4]:
vg = jax.value_and_grad(infidelity)(params, target_unitary)
print('The infidelity of simultaneous X gates is ', 10**vg[0])


The infidelity of simultaneous X gates is  0.0006533218397565268


In [5]:
print('The gradient of log10(infidelity):')
tree_print(vg[1])


The gradient of log10(infidelity):
{ 'capacitive_coupling_all_unify': {'strength': -0.30678138960742607},
  'inductive_coupling_all_unify': {'strength': -158.6583899707822},
  'q02': { 'ec': 0.4434989549839017,
           'ej': 0.12670299682426123,
           'el': -0.1462045689537277},
  'q02_pulse_rampcos': { 'amp': -2.2086894895044833,
                         'omega_d': 5.41994775487869,
                         'phase': 0.181825493321193,
                         't_plateau': 0.00035738436408789973,
                         't_ramp': 0.0007715123434355182},
  'q03': { 'ec': -13.55767252401893,
           'ej': 4.085588336631121,
           'el': -12.431915770031264},
  'q12': { 'ec': -6.45063715091898,
           'ej': 2.017487089770461,
           'el': -5.852363732422589},
  'q12_pulse_rampcos': { 'amp': -0.03347321995285995,
                         'omega_d': 4.2612528500534435,
                         'phase': 0.11020943732382167,
                         't_plateau': -0.004

Show the optimization procedure of control parameters

In [6]:
adam_opt(infidelity,
            params,
            args=(target_unitary,),
            options={
                'adam_lr': 1e-6,
                'steps': 3
            })


{'adam_lr': 1e-06, 'adam_lr_decay_rate': 1000, 'steps': 3, 'adam_b1': 0.9, 'adam_b2': 0.999}
step: 0
parameters:
{ 'capacitive_coupling_all_unify': {'strength': 0.07225663},
  'inductive_coupling_all_unify': {'strength': -0.01256637},
  'q02': {'ec': 6.3532393823, 'ej': 25.1103087675, 'el': 6.344787656},
  'q02_pulse_rampcos': { 'amp': 0.18128846,
                         'omega_d': 2.58934559,
                         'phase': -0.24290228,
                         't_plateau': 69.93608145,
                         't_ramp': 29.92806488},
  'q03': {'ec': 6.2550610547000005, 'ej': 25.1411401038, 'el': 5.000964059},
  'q12': {'ec': 6.2233456874, 'ej': 25.1512697752, 'el': 5.6012793457},
  'q12_pulse_rampcos': { 'amp': 0.17872194,
                         'omega_d': 4.19989714,
                         'phase': -0.01543561,
                         't_plateau': 69.95327862,
                         't_ramp': 29.94562879},
  'q13': {'ec': 6.2867872105, 'ej': 25.1316550509, 'el': 6.91445299

{'capacitive_coupling_all_unify': {'strength': Array(0.07225774, dtype=float64)},
 'inductive_coupling_all_unify': {'strength': Array(-0.01256337, dtype=float64)},
 'q02': {'ec': Array(6.35323779, dtype=float64),
  'ej': Array(25.11030808, dtype=float64),
  'el': Array(6.34478826, dtype=float64)},
 'q02_pulse_rampcos': {'amp': Array(0.1812913, dtype=float64),
  'omega_d': Array(2.58934343, dtype=float64),
  'phase': Array(-0.24290522, dtype=float64),
  't_plateau': Array(69.93608186, dtype=float64),
  't_ramp': Array(29.92806516, dtype=float64)},
 'q03': {'ec': Array(6.25506345, dtype=float64),
  'ej': Array(25.1411377, dtype=float64),
  'el': Array(5.00096646, dtype=float64)},
 'q12': {'ec': Array(6.22334632, dtype=float64),
  'ej': Array(25.15126912, dtype=float64),
  'el': Array(5.60127999, dtype=float64)},
 'q12_pulse_rampcos': {'amp': Array(0.17872454, dtype=float64),
  'omega_d': Array(4.19989418, dtype=float64),
  'phase': Array(-0.01543842, dtype=float64),
  't_plateau': Array(